### Assignment 10.1

Write a class that determines the proximity of users and objects based on SVD decomposition.

This requires the implementation of several functions, which are titled in the class template below.


    1. _get_svd() function must perform the svd transformation and return the P and Q matrices (see the lecture).

    2. get_similar_users() should rank all users in order of proximity to the searched user and return the indexes of the closest ones.

    3. get_similar_items() - similar, but with respect to objects. As arguments, the last two functions take (1) the number of similar users and objects whose indexes to return, and (2) the index of the current user or object.

#### Solution

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
from numpy.linalg import svd

from sklearn.neighbors import NearestNeighbors

In [2]:
class similaryty_analizer(object):
    def __init__(self, R: np.array):
        self.R = R
        self.n_users = None
        self.n_items = None

    def _get_svd(self, new_dim: int):
        R = self.R
        U, S, V = svd(R)

        U = U[:, :new_dim]
        S = np.diag(S[:new_dim])
        V = V[:, :new_dim]

        P = np.matmul(U, S)
        Q = V
        return P, Q

    def get_similar_users(self, n_users: int, user_id: int):
        self.n_users = n_users
        P, _ = self._get_svd(n_users)
        user = P[user_id, :]
    
        nn = NearestNeighbors(n_neighbors=n_users+1)        
        nn = nn.fit(P)
        
        _, neighbours = nn.kneighbors(user.reshape(1, -1), n_users)
        neighbours = neighbours[0, :]
        
        for i in range(n_users):
            if neighbours[i] >= user_id:
                neighbours[i] += 1
                
        return neighbours

    def get_similar_items(self, n_items: int, item_id: int):
        self.n_items = n_items
        _, Q = self._get_svd(n_items)
        item = Q[item_id, :]
        Q = np.delete(Q,(item_id), axis = 0)
        
        nn = NearestNeighbors(n_neighbors=n_items+1)
        nn = nn.fit(Q)
        
        _, neighbours = nn.kneighbors(item.reshape(1, -1), n_items)
        neighbours = neighbours[0, :]
        
        for i in range(n_items):
            if neighbours[i] >= item_id:
                neighbours[i] += 1
         
        return neighbours

In [3]:
r1 = [1, 0, 0]
r2 = [1, 1, 0]
r3 = [1, 1, 1]

R = np.array([r1, r2, r3])

SA = similaryty_analizer(R)
n = SA.get_similar_users(1, 0)
n

array([1])